In [16]:
import vectorbt as vbt
import yfinance
import datetime
import pandas as pd
import numpy as np

In [111]:
end_time = datetime.datetime.now()
start_time = end_time - datetime.timedelta(days=2)

In [112]:
btc_price = vbt.YFData.download(
    'ETH-USD',
    missing_index = 'drop',
    start = start_time,
    end = end_time,
    interval= "1m").get('Close')

In [113]:
btc_price.head()

Datetime
2023-10-01 05:25:00+00:00    1678.689941
2023-10-01 05:26:00+00:00    1678.640625
2023-10-01 05:27:00+00:00    1678.856812
2023-10-01 05:28:00+00:00    1679.246094
2023-10-01 05:29:00+00:00    1679.428589
Name: Close, dtype: float64

In [21]:
def custom_indicator(close,window=14):
    rsi = vbt.RSI.run(close,window=window)
    return rsi.rsi

In [23]:
ind = vbt.IndicatorFactory(
    class_name = "Combination",
    short_name = 'comb',
    input_names= ['close'],
    param_names=  ['window'],
    output_names = ['value']
    ).from_apply_func(
                custom_indicator,
                window = 14
                )

In [24]:
res = ind.run(
    btc_price,
    window = 21)

In [25]:
res.value

Datetime
2023-09-30 05:16:00+00:00          NaN
2023-09-30 05:17:00+00:00          NaN
2023-09-30 05:18:00+00:00          NaN
2023-09-30 05:19:00+00:00          NaN
2023-09-30 05:20:00+00:00          NaN
                               ...    
2023-10-02 05:08:00+00:00    44.328440
2023-10-02 05:09:00+00:00    50.846761
2023-10-02 05:10:00+00:00    52.700563
2023-10-02 05:11:00+00:00    54.149025
2023-10-02 05:12:00+00:00    46.876559
Name: (21, Close), Length: 2869, dtype: float64

In [42]:
def custom_indicator(close, rsi_window = 14, ma_window= 50):
    rsi = vbt.RSI.run(close, window = rsi_window).rsi
    ma = vbt.MA.run(close, ma_window).ma
    return rsi

In [43]:
ind = vbt.IndicatorFactory(
    class_name='Combination',
    short_name='comb',
    input_names=['close'],
    param_names=['rsi_window','ma_window'],
    output_names=['value']
    ).from_apply_func(
            custom_indicator,
            rsi_window = 14,
            ma_window = 50
            )

In [44]:
res = ind.run(
    btc_price,
    rsi_window = 21,
    ma_window = 50)

In [47]:
print(res.value)

Datetime
2023-09-30 05:16:00+00:00          NaN
2023-09-30 05:17:00+00:00          NaN
2023-09-30 05:18:00+00:00          NaN
2023-09-30 05:19:00+00:00          NaN
2023-09-30 05:20:00+00:00          NaN
                               ...    
2023-10-02 05:08:00+00:00    44.328440
2023-10-02 05:09:00+00:00    50.846761
2023-10-02 05:10:00+00:00    52.700563
2023-10-02 05:11:00+00:00    54.149025
2023-10-02 05:12:00+00:00    46.876559
Name: (21, 50, Close), Length: 2869, dtype: float64


In [54]:
def custom_indicator(close, rsi_window = 14, ma_window= 50):
    rsi = vbt.RSI.run(close, window = rsi_window).rsi.to_numpy()
    ma = vbt.MA.run(close, ma_window).ma.to_numpy()
    trend = np.where(rsi > 70, -1, 0)
    trend = np.where(rsi < 30, 1, trend)
    return trend

In [55]:
ind = vbt.IndicatorFactory(
    class_name='Combination',
    short_name='comb',
    input_names=['close'],
    param_names=['rsi_window','ma_window'],
    output_names=['value']
    ).from_apply_func(
            custom_indicator,
            rsi_window = 14,
            ma_window = 50
            )

In [56]:
res = ind.run(
    btc_price,
    rsi_window = 21,
    ma_window = 50)

In [64]:
print(res.value.unique())

[ 0 -1  1]


In [65]:
def custom_indicator(close, rsi_window = 14, ma_window= 50):
    rsi = vbt.RSI.run(close, window = rsi_window).rsi.to_numpy()
    ma = vbt.MA.run(close, ma_window).ma.to_numpy()
    trend = np.where(rsi > 70, -1, 0)
    trend = np.where((rsi < 30) & (close < ma), 1, trend)
    return trend

In [66]:
ind = vbt.IndicatorFactory(
    class_name='Combination',
    short_name='comb',
    input_names=['close'],
    param_names=['rsi_window','ma_window'],
    output_names=['value']
    ).from_apply_func(
            custom_indicator,
            rsi_window = 14,
            ma_window = 50
            )

In [67]:
res = ind.run(
    btc_price,
    rsi_window = 21,
    ma_window = 50)

print(res.value.unique())

[ 0 -1  1]


In [68]:
entries = res.value == 1.0
exits = res.value == -1.0

In [69]:
pf = vbt.Portfolio.from_signals(btc_price,entries,exits)

In [71]:
pf.stats()

C:\Users\HP\anaconda3\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning: Metric 'sharpe_ratio' requires frequency to be set
  warnings.warn(warning_message)
C:\Users\HP\anaconda3\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning: Metric 'calmar_ratio' requires frequency to be set
  warnings.warn(warning_message)
C:\Users\HP\anaconda3\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning: Metric 'omega_ratio' requires frequency to be set
  warnings.warn(warning_message)
C:\Users\HP\anaconda3\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning: Metric 'sortino_ratio' requires frequency to be set
  warnings.warn(warning_message)


Start                         2023-09-30 05:16:00+00:00
End                           2023-10-02 05:12:00+00:00
Period                                             2869
Start Value                                       100.0
End Value                                    100.294937
Total Return [%]                               0.294937
Benchmark Return [%]                           3.236058
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                               0.998873
Max Drawdown Duration                            1084.0
Total Trades                                         24
Total Closed Trades                                  23
Total Open Trades                                     1
Open Trade PnL                                -0.068254
Win Rate [%]                                  65.217391
Best Trade [%]                                 0.192095
Worst Trade [%]                               -0

In [117]:
btc_price = vbt.YFData.download(
    ['BTC-USD','ETH-USD'],
    missing_index = 'drop',
    start = start_time,
    end = end_time,
    interval= "1m").get('Close')

In [128]:
def custom_indicator(close, rsi_window = 14, ma_window= 50):
    close_5m = close.resample('5T').last()
    rsi = vbt.RSI.run(close_5m, window = rsi_window).rsi
    rsi, _ = rsi.align(close,
                       broadcast_axis =0,
                       method = 'ffill',
                       join= 'right')
    close = close.to_numpy()
    rsi = rsi.to_numpy()
    ma = vbt.MA.run(close, ma_window).ma.to_numpy()
    trend = np.where(rsi > 70, -1, 0)
    trend = np.where((rsi < 30) & (close < ma), 1, trend)
    return trend

In [129]:
ind = vbt.IndicatorFactory(
    class_name='Combination',
    short_name='comb',
    input_names=['close'],
    param_names=['rsi_window','ma_window'],
    output_names=['value']
    ).from_apply_func(
            custom_indicator,
            rsi_window = 14,
            ma_window = 50,
            keep_pd = True
            )

In [130]:
res = ind.run(
    btc_price,
    rsi_window = 21,
    ma_window = 50)

In [131]:
res.value

comb_rsi_window                21        
comb_ma_window                 50        
symbol                    BTC-USD ETH-USD
Datetime                                 
2023-10-01 05:25:00+00:00       0       0
2023-10-01 05:26:00+00:00       0       0
2023-10-01 05:27:00+00:00       0       0
2023-10-01 05:28:00+00:00       0       0
2023-10-01 05:29:00+00:00       0       0
...                           ...     ...
2023-10-03 05:19:00+00:00       0       0
2023-10-03 05:20:00+00:00       0       0
2023-10-03 05:21:00+00:00       0       0
2023-10-03 05:22:00+00:00       0       0
2023-10-03 05:23:00+00:00       0       0

[2874 rows x 2 columns]

In [132]:
entries = res.value == 1.0
exits = res.value ==-1.0
pf = vbt.Portfolio.from_signals(btc_price,entries, exits)
print(pf.total_return())

comb_rsi_window  comb_ma_window  symbol 
21               50              BTC-USD   -0.003428
                                 ETH-USD   -0.027500
Name: total_return, dtype: float64
